# Generating artificial data

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os


In [4]:
import numpy as np
import os

np.random.seed(1234)  # For reproducibility
n_series = 4000   # Number of time series per class
n_points = 500    # Number of data points in each time series

#output_dir = 'time_series_data'
#os.makedirs(output_dir, exist_ok=True)

output_dir = os.path.expanduser("~/timeseries_data")  # Creates the directory in your home folder
os.makedirs(output_dir, exist_ok=True)


### AR, MA and ARMA Model

In [6]:
# Function to generate AR, MA, or ARMA data with optional trend and seasonality
def generate_time_series(model_type, order, n_points, trend_strength=0.1, seasonality_amplitude=0.5, seasonality_period=50, include_trend=True):
   if model_type == 'AR':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -params]
       ma = np.array([1])
   elif model_type == 'MA':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.array([1])
       ma = np.r_[1, params]
   elif model_type == 'ARMA':
       ar_params = np.random.uniform(-0.5, 0.5, size=order)
       ma_params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -ar_params]
       ma = np.r_[1, ma_params]
   else:
       raise ValueError("Invalid model type. Use 'AR', 'MA', or 'ARMA'.")
   
   
      # Generate the process
   process = sm.tsa.ArmaProcess(ar, ma)
   data = process.generate_sample(nsample=n_points)
   if include_trend:
       trend = np.linspace(0, trend_strength * n_points, n_points)
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += trend + seasonality
   else:
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += seasonality
   return data

### Models for orders 1-3 with and without trend (Kernel crashed after 14 classes)

In [15]:
# Loop to generate and save time series plots for each class
model_types = ['AR', 'MA', 'ARMA']
orders = [1, 2, 3]
for model_type in model_types:
   for order in orders:
       for include_trend in [True, False]:
           class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
           class_dir = os.path.join(output_dir, class_label)
           os.makedirs(class_dir, exist_ok=True)
           for i in range(n_series):
               data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
               # Plotting the time series
               plt.figure(figsize=(8, 4))
               plt.plot(data)
               plt.axis('off')  # Turn off axes for a clean image
               plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
               plt.close()
print("Time series generation completed. Time series are saved in the 'time_series_data' directory.")

: 

### Generating image for remaining classes

In [7]:
# Adjusted classes for ARMA 2 and 3 with and without trend
model_type = 'ARMA'
orders = [2, 3]
for order in orders:
    for include_trend in [True, False]:
        class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
        class_dir = os.path.join(output_dir, class_label)
        os.makedirs(class_dir, exist_ok=True)
        for i in range(n_series):
            data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
            # Plotting the time series
            plt.figure(figsize=(8, 4))
            plt.plot(data)
            plt.axis('off')  # Turn off axes for a clean image
            plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
            plt.close()

# Code to check the number of images in each folder
folder_status = {}
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path):
        folder_status[folder_name] = len(os.listdir(folder_path))

# Display folder status
for class_label, image_count in folder_status.items():
    print(f"{class_label}: {image_count} images")

ARMA_1_without_trend: 4000 images
ARMA_2_with_trend: 4000 images
MA_3_with_trend: 4000 images
ARMA_2_without_trend: 4000 images
AR_3_without_trend: 4000 images
MA_1_without_trend: 4000 images
MA_1_with_trend: 4000 images
AR_2_with_trend: 4000 images
MA_2_without_trend: 4000 images
AR_1_with_trend: 4000 images
MA_2_with_trend: 4000 images
MA_3_without_trend: 4000 images
ARMA_3_with_trend: 4000 images
AR_3_with_trend: 4000 images
AR_2_without_trend: 4000 images
ARMA_3_without_trend: 4000 images
AR_1_without_trend: 4000 images
ARMA_1_with_trend: 4000 images


# Loading the data

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import SeparableConv2D
from sklearn.utils import shuffle

In [2]:
# Parameters
# Global parameters
image_size = (224, 224)  # Resize all images to this size ->>>>> later time when training model
# batch_size = 32  # Adjust based on hardware capacity

main_dir = os.path.expanduser("~/timeseries_data")  # Path to your main directory with class subfolders


def load_data(main_dir, image_size):
    data = []
    labels = []
    classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])  # Filter directories only
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # Mapping class names to indices

    for cls in classes:
        class_dir = os.path.join(main_dir, cls)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image file
                img = Image.open(img_path).convert('RGB')  # Convert to RGB
                img = img.resize(image_size)  # Resize image
                data.append(np.array(img))
                labels.append(class_to_idx[cls])

    return np.array(data), np.array(labels), classes




In [3]:
# Generate a mapping for classes
classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # {'AR_1_with_trend': 0, 'MA_1_without_trend': 1, ...}
print(f"Class to index mapping: {class_to_idx}")


Class to index mapping: {'ARMA_1_with_trend': 0, 'ARMA_1_without_trend': 1, 'ARMA_2_with_trend': 2, 'ARMA_2_without_trend': 3, 'ARMA_3_with_trend': 4, 'ARMA_3_without_trend': 5, 'AR_1_with_trend': 6, 'AR_1_without_trend': 7, 'AR_2_with_trend': 8, 'AR_2_without_trend': 9, 'AR_3_with_trend': 10, 'AR_3_without_trend': 11, 'MA_1_with_trend': 12, 'MA_1_without_trend': 13, 'MA_2_with_trend': 14, 'MA_2_without_trend': 15, 'MA_3_with_trend': 16, 'MA_3_without_trend': 17}


In [4]:
# Load the dataset
print("Loading dataset...")
data, labels, classes = load_data(main_dir, image_size)

# Normalize data
data = data / 255.0  # Normalize pixel values to [0, 1]

# Print class information
print(f"Classes: {classes}")
print(f"Number of classes: {len(classes)}")


Loading dataset...
Classes: ['ARMA_1_with_trend', 'ARMA_1_without_trend', 'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', 'ARMA_3_without_trend', 'AR_1_with_trend', 'AR_1_without_trend', 'AR_2_with_trend', 'AR_2_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_1_with_trend', 'MA_1_without_trend', 'MA_2_with_trend', 'MA_2_without_trend', 'MA_3_with_trend', 'MA_3_without_trend']
Number of classes: 18


In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

# Subset of classes to start with (e.g., 8 representative classes)
subset_classes = ['AR_1_with_trend', 'AR_2_without_trend', 
                  'MA_2_with_trend', 'MA_3_without_trend',
                  'ARMA_1_with_trend']

# Map subset classes to indices
subset_indices = [class_to_idx[cls] for cls in subset_classes]

# Filter data and labels for the subset
subset_mask = np.isin(labels, subset_indices)
data_subset = data[subset_mask]
labels_subset = labels[subset_mask]

# Reindex labels for the subset
labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])


data_subset = data_subset.astype('float32')

# 'MA_3_without_trend', 'AR_3_with_trend', 'ARMA_2_without_trend', 'ARMA_1_with_trend'




In [6]:
print(f"Data subset shape: {data_subset.shape}, size: {data_subset.nbytes / (1024**2):.2f} MB")
print(f"Labels subset shape: {labels_subset.shape}")


Data subset shape: (20000, 224, 224, 3), size: 11484.38 MB
Labels subset shape: (20000,)


In [7]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import Sequence


# Split indices instead of the whole dataset
train_idx, temp_idx = train_test_split(
    np.arange(len(labels_subset)), test_size=0.3, stratify=labels_subset, random_state=42
)

val_idx, test_idx = train_test_split(
    temp_idx, test_size=1/3, stratify=labels_subset[temp_idx], random_state=42
)

# Custom DataGenerator that uses indices
class IndexedDataGenerator(Sequence):
    def __init__(self, data, labels, indices, batch_size):
        self.data = data
        self.labels = labels
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_data = self.data[batch_indices]
        batch_labels = self.labels[batch_indices]
        return batch_data, batch_labels

# Use indices to create separate generators
train_generator = IndexedDataGenerator(data_subset, labels_subset, train_idx, batch_size=32)
val_generator = IndexedDataGenerator(data_subset, labels_subset, val_idx, batch_size=32)
test_generator = IndexedDataGenerator(data_subset, labels_subset, test_idx, batch_size=32)


In [47]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Parameters
IMAGE_SIZE = (224, 224, 3)  # Input size for ResNet50
NUM_CLASSES = len(subset_classes)  # Number of classes in the subset
BATCH_SIZE = 32
EPOCHS = 20

# Load the ResNet50 model with pretrained weights
base_model = ResNet50(
    input_shape=IMAGE_SIZE,
    include_top=False,  # Exclude the final classification layers
    weights="imagenet"
)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pool the feature maps to a single vector
x = BatchNormalization()(x)  # Normalize for stability
x = Dense(512, activation="relu")(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)  # Final classification layer

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Use a lower learning rate for transfer learning
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Define callbacks
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "subset_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model on the subset
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


Epoch 1/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 735ms/step - accuracy: 0.3980 - loss: 1.1213
Epoch 1: val_accuracy improved from -inf to 0.44025, saving model to subset_model.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 415s 941ms/step - accuracy: 0.3980 - loss: 1.1212 - val_accuracy: 0.4403 - val_loss: 0.9981 - learning_rate: 1.0000e-04
Epoch 2/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 709ms/step - accuracy: 0.4296 - loss: 0.9796
Epoch 2: val_accuracy improved from 0.44025 to 0.46600, saving model to subset_model.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 401s 915ms/step - accuracy: 0.4296 - loss: 0.9796 - val_accuracy: 0.4660 - val_loss: 0.9305 - learning_rate: 1.0000e-04
Epoch 3/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 734ms/step - accuracy: 0.4342 - loss: 0.9566
Epoch 3: val_accuracy improved from 0.46600 to 0.48050, saving model to subset_model.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 416s 951ms/step - accuracy: 0.4342 - loss: 0.9566 - val_accuracy: 0.4805 - val_loss: 0.9207 - learning_rate: 1.0000e-04
Epoch 4/20
438/438 

In [51]:
# Unfreeze some layers in the base model for fine-tuning
for layer in base_model.layers[-50:]:  # Unfreeze the last 50 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
history_finetune = model.fit(
    train_generator,  # Use the same training data
    validation_data=val_generator,
    epochs=15,  # Fine-tune for fewer epochs to avoid overfitting
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


Epoch 1/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4176 - loss: 1.0157
Epoch 1: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━━━━━━━━━━━ 866s 2s/step - accuracy: 0.4176 - loss: 1.0157 - val_accuracy: 0.4375 - val_loss: 0.9349 - learning_rate: 1.0000e-05
Epoch 2/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4503 - loss: 0.9345
Epoch 2: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━━━━━━━━━━━ 862s 2s/step - accuracy: 0.4503 - loss: 0.9345 - val_accuracy: 0.4415 - val_loss: 0.9331 - learning_rate: 1.0000e-05
Epoch 3/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4747 - loss: 0.9176
Epoch 3: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━━━━━━━━━━━ 861s 2s/step - accuracy: 0.4746 - loss: 0.9176 - val_accuracy: 0.4600 - val_loss: 0.9086 - learning_rate: 1.0000e-05
Epoch 4/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4951 - loss: 0.8946
Epoch 4: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━

# nor running cus new dataset function below 

In [ ]:
from tensorflow.keras.models import load_model



# Load the pre-trained model
model = load_model("subset_model.keras")  # Load your pre-trained model with the initial 5 classes

def update_dataset(data, labels, class_to_idx, subset_classes, batch_size=32):
    # Map subset classes to indices
    subset_indices = [class_to_idx[cls] for cls in subset_classes]

    # Filter data and labels for the subset
    subset_mask = np.isin(labels, subset_indices)
    data_subset = data[subset_mask]
    labels_subset = labels[subset_mask]

    # Reindex labels for the subset
    labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])

    # Split data into train, validation, and test indices
    train_idx, temp_idx = train_test_split(
        np.arange(len(labels_subset)), test_size=0.3, stratify=labels_subset, random_state=42
    )
    val_idx, test_idx = train_test_split(
        temp_idx, test_size=1/3, stratify=labels_subset[temp_idx], random_state=42
    )

    # Create new generators
    train_generator = IndexedDataGenerator(data_subset, labels_subset, train_idx, batch_size=batch_size)
    val_generator = IndexedDataGenerator(data_subset, labels_subset, val_idx, batch_size=batch_size)

    return train_generator, val_generator, data_subset, labels_subset

# need to run

In [8]:

def update_model(model, num_classes, learning_rate=1e-5):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense

    # Modify the output layer for the new number of classes
    new_output = Dense(num_classes, activation="softmax")(model.layers[-3].output)
    updated_model = Model(inputs=model.input, outputs=new_output)

    # Recompile the model with a smaller learning rate
    updated_model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return updated_model

# not running new function below

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


def incremental_training(
    model,
    data,
    labels,
    class_to_idx,
    current_classes,
    new_classes,
    batch_size=32,
    epochs=10,
    learning_rate=1e-5,
    checkpoint_name="incremental_model"
):
    # Create a copy of current_classes and add new classes to it
    updated_classes = current_classes + new_classes

    # Update dataset and generators
    train_generator, val_generator, _, _ = update_dataset(data, labels, class_to_idx, updated_classes, batch_size)

    # Update the model
    updated_model = update_model(model, len(updated_classes), learning_rate)

    # Define callbacks
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    checkpoint = ModelCheckpoint(f"{checkpoint_name}_{len(updated_classes)}_classes.keras", monitor="val_accuracy", save_best_only=True, verbose=1)

    # Train the model
    history = updated_model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[early_stopping, reduce_lr, checkpoint],
        verbose=1
    )

    return updated_model, history, updated_classes

# kernel crashed after the first increment -> model 8 classes saved tho

In [ ]:

# Initialize the current subset of classes
current_classes = ['AR_1_with_trend', 'AR_2_without_trend', 'MA_2_with_trend', 'MA_3_without_trend', 'ARMA_1_with_trend']

# Define the new classes to add incrementally
additional_classes = [
    ['AR_1_without_trend', 'MA_1_with_trend', 'MA_1_without_trend'], # First increment
    ['AR_2_with_trend', 'MA_2_without_trend', 'ARMA_1_without_trend'], # Second increment
    ['AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend'], # Third increment
    ['ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', 'ARMA_3_without_trend'] # Final increment (4 classes)
]

# Train the model incrementally
for new_classes in additional_classes:
    model, history, current_classes = incremental_training(
        model=model,
        data=data,
        labels=labels,
        class_to_idx=class_to_idx,
        current_classes=current_classes,
        new_classes=new_classes,
        batch_size=32,
        epochs=10,
        learning_rate=1e-5,
        checkpoint_name="incremental_model"
    )

In [9]:
from tensorflow.keras.models import load_model

# Load the saved model after the first increment
#model = load_model("incremental_model_8_classes.keras")

# Define the current classes from the first increment
current_classes = [
    'AR_1_with_trend', 'AR_2_without_trend',
    'MA_2_with_trend', 'MA_3_without_trend',
    'ARMA_1_with_trend', 'AR_1_without_trend',
    'MA_1_with_trend', 'MA_1_without_trend',
    'AR_2_with_trend', 'MA_2_without_trend', 'ARMA_1_without_trend', 
    'AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend',
]


In [10]:
# Remaining classes to train
remaining_classes = [
    #'AR_2_with_trend', 'MA_2_without_trend', 'ARMA_1_without_trend',  # Increment 1
    #'AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend',       # Increment 2
    'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', # Increment 3
    'ARMA_3_without_trend'  # Increment 4
]

# Split into smaller increments
increment_1 = remaining_classes[:3]  # First batch of 3 classes
increment_2 = remaining_classes[3:6]  # Second batch of 3 classes
increment_3 = remaining_classes[6:9]  # Third batch of 3 classes
increment_4 = remaining_classes[9:]   # Fourth batch of 1 class



In [11]:
def update_dataset(data, labels, class_to_idx, remembered_classes, new_classes, batch_size=32):
    # Combine remembered and new classes
    subset_classes = remembered_classes + new_classes
    subset_indices = [class_to_idx[cls] for cls in subset_classes]
    subset_mask = np.isin(labels, subset_indices)
    data_subset = data[subset_mask]
    labels_subset = labels[subset_mask]
    labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])

    # Split data into train, validation, and test indices
    train_idx, temp_idx = train_test_split(
        np.arange(len(labels_subset)), test_size=0.3, stratify=labels_subset, random_state=42
    )
    val_idx, test_idx = train_test_split(
        temp_idx, test_size=1/3, stratify=labels_subset[temp_idx], random_state=42
    )

    # Create new generators
    train_generator = IndexedDataGenerator(data_subset, labels_subset, train_idx, batch_size=batch_size)
    val_generator = IndexedDataGenerator(data_subset, labels_subset, val_idx, batch_size=batch_size)

    return train_generator, val_generator, data_subset, labels_subset


In [12]:
def update_model(model, num_classes, learning_rate=1e-5):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.regularizers import l2

    # Modify the output layer for the new number of classes
    x = model.layers[-3].output  # Access the layer before the final layer
    x = BatchNormalization()(x)
    x = Dense(512, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    new_output = Dense(num_classes, activation="softmax")(x)

    # Create the updated model
    updated_model = Model(inputs=model.input, outputs=new_output)

    # Recompile the model
    updated_model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return updated_model


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)


# not running alreday have saved model

In [14]:
# Train the model on increment 1
new_classes = increment_1  # First batch of 3 new classes

# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,
    labels=labels,
    class_to_idx=class_to_idx,
    remembered_classes=current_classes[-3:],  # Remember the first 3 classes
    new_classes=new_classes,
    batch_size=32
)

# Update the model
#model = update_model(model, num_classes=len(current_classes[:3]) + len(new_classes), learning_rate=1e-5)


model = update_model(
    model, 
    num_classes=len(current_classes[-3:]) + len(new_classes),  # 8 existing + 3 new = 11
    learning_rate=1e-5
)

# Define checkpoint
checkpoint = ModelCheckpoint(
    f"incremental_model_{len(current_classes[-3:]) + len(new_classes)}_classes.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update current classes
current_classes = (current_classes[-3:]) + new_classes


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3155 - loss: 7.2498
Epoch 1: val_accuracy improved from -inf to 0.33542, saving model to incremental_model_6_classes.keras
525/525 ━━━━━━━━━━━━━━━━━━━━ 1172s 2s/step - accuracy: 0.3155 - loss: 7.2454 - val_accuracy: 0.3354 - val_loss: 1.9726 - learning_rate: 0.0010
Epoch 2/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3435 - loss: 1.6941
Epoch 2: val_accuracy did not improve from 0.33542
525/525 ━━━━━━━━━━━━━━━━━━━━ 1132s 2s/step - accuracy: 0.3435 - loss: 1.6938 - val_accuracy: 0.3333 - val_loss: 3.0074 - learning_rate: 0.0010
Epoch 3/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3525 - loss: 1.3137
Epoch 3: val_accuracy did not improve from 0.33542
525/525 ━━━━━━━━━━━━━━━━━━━━ 1224s 2s/step - accuracy: 0.3525 - loss: 1.3137 - val_accuracy: 0.1667 - val_loss: 7.1886 - learning_rate: 0.0010
Epoch 4/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3482 - loss: 1.2848
Epoch 4: val_accuracy impr

In [14]:
from tensorflow.keras.models import load_model

# Load the model saved after increment 1
model = load_model("incremental_model_6_classes.keras")  # Model from increment 1


In [ ]:
def update_model(model, num_classes, learning_rate=1e-5):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.regularizers import l2

    # Modify the output layer for the new number of classes
    x = model.layers[-3].output  # Access the layer before the final layer
    x = BatchNormalization(name="batch_norm_update_1")(x)  # Unique name for the layer
    x = Dense(512, activation="relu", kernel_regularizer=l2(0.01), name="dense_update_1")(x)  # Unique name
    x = Dropout(0.5, name="dropout_update_1")(x)  # Unique name
    x = BatchNormalization(name="batch_norm_update_2")(x)  # Unique name
    x = Dense(256, activation="relu", kernel_regularizer=l2(0.01), name="dense_update_2")(x)  # Unique name
    x = Dropout(0.5, name="dropout_update_2")(x)  # Unique name
    new_output = Dense(num_classes, activation="softmax", name="dense_output_update")(x)  # Unique name

    # Create the updated model
    updated_model = Model(inputs=model.input, outputs=new_output)

    # Recompile the model
    updated_model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return updated_model


In [16]:
# Use increment 2 for the second batch of new classes
new_classes = increment_2  # ['AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend']


# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,
    labels=labels,
    class_to_idx=class_to_idx,
    remembered_classes=current_classes[-3:],  # Remember the first 3 classes
    new_classes=new_classes,
    batch_size=32
)

# Update the model
model = update_model(
    model, 
    num_classes=len(current_classes[-3:]) + len(new_classes),  # 8 existing + 3 new = 11
    learning_rate=1e-5
)

# Define checkpoint
checkpoint = ModelCheckpoint(
    f"incremental_model_{len(current_classes[-3:]) + len(new_classes)}_classes.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update current classes
current_classes = (current_classes[-3:]) + new_classes


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3446 - loss: 6.1893
Epoch 1: val_accuracy improved from -inf to 0.33375, saving model to incremental_model_6_classes.keras
525/525 ━━━━━━━━━━━━━━━━━━━━ 1100s 2s/step - accuracy: 0.3446 - loss: 6.1860 - val_accuracy: 0.3338 - val_loss: 3.1009 - learning_rate: 0.0010
Epoch 2/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4173 - loss: 1.8325
Epoch 2: val_accuracy did not improve from 0.33375
525/525 ━━━━━━━━━━━━━━━━━━━━ 1069s 2s/step - accuracy: 0.4173 - loss: 1.8321 - val_accuracy: 0.1444 - val_loss: 109.7172 - learning_rate: 0.0010
Epoch 3/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4444 - loss: 1.3133
Epoch 3: val_accuracy improved from 0.33375 to 0.36125, saving model to incremental_model_6_classes.keras
525/525 ━━━━━━━━━━━━━━━━━━━━ 1138s 2s/step - accuracy: 0.4444 - loss: 1.3132 - val_accuracy: 0.3613 - val_loss: 30.8598 - learning_rate: 0.0010
Epoch 4/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - 

In [13]:
from tensorflow.keras.models import load_model

# Load the model saved after increment 2
model = load_model("incremental_model_6_classes.keras")  # Model from increment 2


In [16]:
def update_model(model, num_classes, learning_rate=1e-5):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.regularizers import l2

    # Modify the output layer for the new number of classes
    x = model.layers[-3].output  # Access the layer before the final layer
    x = BatchNormalization(name="batch_norm_update_3_1")(x)  # Unique name for the layer
    x = Dense(512, activation="relu", kernel_regularizer=l2(0.01), name="dense_update_3_1")(x)  # Unique name
    x = Dropout(0.5, name="dropout_update_3_1")(x)  # Unique name
    x = BatchNormalization(name="batch_norm_update_3_2")(x)  # Unique name
    x = Dense(256, activation="relu", kernel_regularizer=l2(0.01), name="dense_update_3_2")(x)  # Unique name
    x = Dropout(0.6, name="dropout_update_3_2")(x)  # Unique name
    new_output = Dense(num_classes, activation="softmax", name="dense_output_update_3")(x)  # Unique name

    # Create the updated model
    updated_model = Model(inputs=model.input, outputs=new_output)

    # Recompile the model
    updated_model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return updated_model


In [17]:
# Use increment 3 for the second batch of new classes
new_classes = increment_3 

# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,
    labels=labels,
    class_to_idx=class_to_idx,
    remembered_classes=current_classes[-3:],  # Remember the first 3 classes
    new_classes=new_classes,
    batch_size=32
)

# Update the model
model = update_model(
    model, 
    num_classes=len(current_classes[-3:]) + len(new_classes), 
    learning_rate=1e-5
)

# Define checkpoint
checkpoint = ModelCheckpoint(
    f"incremental_model_{len(current_classes[-3:]) + len(new_classes)}_classes.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update current classes
current_classes = (current_classes[-3:]) + new_classes


Epoch 1/10


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6651 - loss: 5.6487
Epoch 1: val_accuracy improved from -inf to 0.69792, saving model to incremental_model_3_classes.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 544s 2s/step - accuracy: 0.6652 - loss: 5.6432 - val_accuracy: 0.6979 - val_loss: 2.2799 - learning_rate: 0.0010
Epoch 2/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6886 - loss: 1.7692
Epoch 2: val_accuracy did not improve from 0.69792
263/263 ━━━━━━━━━━━━━━━━━━━━ 530s 2s/step - accuracy: 0.6886 - loss: 1.7681 - val_accuracy: 0.4079 - val_loss: 2.2568 - learning_rate: 0.0010
Epoch 3/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7238 - loss: 1.0279
Epoch 3: val_accuracy did not improve from 0.69792
263/263 ━━━━━━━━━━━━━━━━━━━━ 533s 2s/step - accuracy: 0.7238 - loss: 1.0276 - val_accuracy: 0.6550 - val_loss: 1.0346 - learning_rate: 0.0010
Epoch 4/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7157 - loss: 0.8560
Epoch 4: val_accuracy improved from 0.69

In [18]:
from tensorflow.keras.models import load_model

# Load the model saved after increment 3
model = load_model("incremental_model_3_classes.keras")  # Model from increment 3


In [19]:
def update_model(model, num_classes, learning_rate=1e-5):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.regularizers import l2

    # Modify the output layer for the new number of classes
    x = model.layers[-3].output  # Access the layer before the final layer
    x = BatchNormalization(name="batch_norm_update_4_1")(x)  # Unique name for the layer
    x = Dense(512, activation="relu", kernel_regularizer=l2(0.01), name="dense_update_4_1")(x)  # Unique name
    x = Dropout(0.5, name="dropout_update_4_1")(x)  # Unique name
    x = BatchNormalization(name="batch_norm_update_4_2")(x)  # Unique name
    x = Dense(256, activation="relu", kernel_regularizer=l2(0.01), name="dense_update_4_2")(x)  # Unique name
    x = Dropout(0.6, name="dropout_update_4_2")(x)  # Unique name
    new_output = Dense(num_classes, activation="softmax", name="dense_output_update_4")(x)  # Unique name

    # Create the updated model
    updated_model = Model(inputs=model.input, outputs=new_output)

    # Recompile the model
    updated_model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return updated_model


In [20]:
# Use increment 4 for the second batch of new classes
new_classes = increment_4

# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,
    labels=labels,
    class_to_idx=class_to_idx,
    remembered_classes=current_classes[-3:],  # Remember the first 3 classes
    new_classes=new_classes,
    batch_size=32
)

# Update the model
model = update_model(
    model, 
    num_classes=len(current_classes[-3:]) + len(new_classes), 
    learning_rate=1e-5
)

# Define checkpoint
checkpoint = ModelCheckpoint(
    f"incremental_model_{len(current_classes[-3:]) + len(new_classes)}_classes.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update current classes
current_classes = (current_classes[-3:]) + new_classes


Epoch 1/10


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6708 - loss: 6.2032
Epoch 1: val_accuracy improved from -inf to 0.71833, saving model to incremental_model_3_classes.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 555s 2s/step - accuracy: 0.6708 - loss: 6.1973 - val_accuracy: 0.7183 - val_loss: 2.4992 - learning_rate: 0.0010
Epoch 2/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6998 - loss: 2.0560
Epoch 2: val_accuracy did not improve from 0.71833
263/263 ━━━━━━━━━━━━━━━━━━━━ 529s 2s/step - accuracy: 0.6998 - loss: 2.0547 - val_accuracy: 0.7150 - val_loss: 1.1826 - learning_rate: 0.0010
Epoch 3/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6984 - loss: 1.1828
Epoch 3: val_accuracy did not improve from 0.71833
263/263 ━━━━━━━━━━━━━━━━━━━━ 524s 2s/step - accuracy: 0.6984 - loss: 1.1827 - val_accuracy: 0.6821 - val_loss: 0.9673 - learning_rate: 0.0010
Epoch 4/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7057 - loss: 1.2019
Epoch 4: val_accuracy did not improve fr

In [28]:
from tensorflow.keras.models import load_model

# Load the trained model (e.g., from the last increment) i think with 11?
model = load_model("incremental_model_8_classes.keras")  # Replace with the correct filename

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

# Print the results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 46s 705ms/step - accuracy: 0.3463 - loss: 1.4209
Test Loss: 1.3699865341186523
Test Accuracy: 0.3605000078678131


In [29]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("incremental_model_8_classes.keras")

# Check the number of output classes
num_classes = model.output_shape[-1]
print(f"The model was trained on {num_classes} classes.")


The model was trained on 8 classes.


In [31]:
# Reconstruct class names using `class_to_idx`
# Example: class_to_idx = {'class1': 0, 'class2': 1, 'class3': 2}
trained_classes = [class_name for class_name, idx in sorted(class_to_idx.items(), key=lambda x: x[1])]
print(f"The model was trained on the following classes: {trained_classes}")


The model was trained on the following classes: ['ARMA_1_with_trend', 'ARMA_1_without_trend', 'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', 'ARMA_3_without_trend', 'AR_1_with_trend', 'AR_1_without_trend', 'AR_2_with_trend', 'AR_2_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_1_with_trend', 'MA_1_without_trend', 'MA_2_with_trend', 'MA_2_without_trend', 'MA_3_with_trend', 'MA_3_without_trend']


In [32]:
# Assuming `current_classes` was updated after each increment
print(f"The model was trained on these classes: {current_classes}")


The model was trained on these classes: ['AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend']


In [ ]:
from tensorflow.keras.models import load_model

# Load the previously saved model
model = load_model("incremental_model_8_classes.keras")  # Starting from the first increment (8 classes)

# Define remembered classes
remembered_classes = [
    'AR_1_with_trend', 'AR_2_without_trend', 
    'MA_2_with_trend', 'MA_3_without_trend', 
    'ARMA_1_with_trend', 'AR_1_without_trend', 
    'MA_1_with_trend', 'MA_1_without_trend'
]

# Train each increment
increments = [increment_1, increment_2, increment_3]
for i, new_classes in enumerate(increments):
    # Update the dataset for the current increment
    train_generator, val_generator, _, _ = update_dataset(
        data=data,
        labels=labels,
        class_to_idx=class_to_idx,
        remembered_classes=remembered_classes[:5],  # Keep a subset of remembered classes (e.g., 5)
        new_classes=new_classes,
        batch_size=32
    )
    
    # Update the model for the new total number of classes
    model = update_model(model, num_classes=len(remembered_classes[:5]) + len(new_classes), learning_rate=1e-5)
    
    # Define the checkpoint for this increment
    checkpoint = ModelCheckpoint(
        f"incremental_model_{len(remembered_classes[:5]) + len(new_classes)}_classes.keras", 
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
    
    # Train the model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,
        callbacks=[early_stopping, reduce_lr, checkpoint],
        verbose=1
    )
    
    # Update the remembered classes
    remembered_classes += new_classes


In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('model_subset.keras')  # Use the correct file path and format


In [25]:
# Reload the full dataset
data_full = data.astype('float32')  # Use the original dataset
labels_full = labels  # Labels for all classes


: 

In [ ]:
# Full class mapping (already created earlier)
print(f"All classes: {classes}")
print(f"Number of classes: {len(classes)}")


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Define callbacks
checkpoint = ModelCheckpoint('model_subset5.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, verbose=1)

# Train the model on the subset
history_subset = model_subset.fit(train_generator,
                                  validation_data=val_generator,
                                  epochs=20,  # Adjust as needed
                                  callbacks=[checkpoint, reduce_lr],
                                  verbose=1)
